In [1]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

In [37]:
datos = pd.read_csv('online_shoppers_intention.csv')
print(datos.dtypes)
for i,val in enumerate(datos_ok.columns):
    labels = datos_ok[val].astype('category').cat.categories.tolist()
    dict_make = {val : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
    datos_ok[val].replace(dict_make[val], inplace=True)

X_all = np.array(datos_ok.drop(columns = 'Revenue').copy())
y_all = np.array(pd.factorize(datos_ok['Revenue'])[0])
print(X_all.shape)

Administrative             float64
Administrative_Duration    float64
Informational              float64
Informational_Duration     float64
ProductRelated             float64
ProductRelated_Duration    float64
BounceRates                float64
ExitRates                  float64
PageValues                 float64
SpecialDay                 float64
Month                       object
OperatingSystems             int64
Browser                      int64
Region                       int64
TrafficType                  int64
VisitorType                 object
Weekend                       bool
Revenue                       bool
dtype: object
(12330, 17)


In [ ]:
# Primero creamos el DataFrame con las variables categóricas
autos_cat = autos.select_dtypes(include='object')

In [17]:
## Month,VisitorType,Weekend.

Eliminamos los ejemplos con valores desconocidos

In [26]:
mascara = np.isnan(X_all)
mascara = np.sum(n,axis=1).reshape(-1,1).astype("bool")
mascara = n == False

X = X_all[n.flatten()]
y = y_all[n.flatten()]
print('se ha eliminado', len(X_all)-len(X),'ejemplos.')

se ha eliminado 14 ejemplos.


In [27]:
Xpos = X[y == 1]
Xneg = X[y == 0]
print('hay',len(Xpos),'positivos')
print('hay',len(Xneg),'negativos')

hay 1908 positivos
hay 10408 negativos


In [14]:
#hanwei
# Se importan las 3 librerías necesarias para resolver el ejercicio
from sklearn import neighbors, metrics, model_selection
import sklearn.tree as tree

# Se fija la semilla de numpy para que la generación aleatoria siempre nos de los mismos números
np.random.seed(12)

# Lllamada a la función train_test_split y guardado del resultado
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.7)

# Realizamos el proceso para KNN por lo que hay que llamar al constructor de dicho clasificador
clf = tree.DecisionTreeClassifier()

# Llamada a la función que realiza el aprendizaje del clasificador
clf.fit(X_train,y_train)

# Llamada a la función que realiza la predicción de los datos de entrenamiento
predTrain = clf.predict(X_train)

# Llamada a la función que calcula el porcentaje de acierto para los datos de entrenamiento
accTrain = round(metrics.accuracy_score(y_train,predTrain)*100,2)
print('El rendimiento en entrenamiento con todas las variables es el {}%'.format(accTrain))

# Llamada a la función que realiza la predicción de los datos de test
predTest = clf.predict(X_test)

# Llamada a la función que calcula el porcentaje de acierto para los datos de test
accTest = round(metrics.accuracy_score(y_test,predTest)*100,2)
print('El rendimiento en test con todas las variables es el {}%'.format(accTest))

El rendimiento en entrenamiento con todas las variables es el 100.0%
El rendimiento en test con todas las variables es el 85.93%


In [ ]:
def normalizarMediaStd(datos):
    media = datos.mean(axis=0)
    desviacion = datos.std(axis=0)
    
    datos = (datos-media)/desviacion
    return media,desviacion,datos


In [ ]:
def deteccionOutliers(datos,media,desviacion,k=2):
    variablesOutliers = (datos < (media - k*desviacion)) | (datos > (media + k*desviacion))
    mascaraOutliers = variablesOutliers.sum(axis=1,dtype=bool)
    indicesOutliers = np.array(range(len(datos)))[mascaraOutliers]
    return indicesOutliers

In [ ]:
#hanwei


In [ ]:
#jhonny
import 

In [ ]:
#jhonny

In [ ]:
#jhonny